In [9]:
import re
import enum
import numpy as np
import pandas as pd

try:
    from . import config
    from utils import timeit, notice
except ImportError:
    import config
    timeit = lambda func: func
    notice = print


class TrafficType(enum.IntEnum):
    # UNKNOWN = 0
    A = 1
    B = 2
    C = 3
    # D = 4
    # E = 5

class TrafficModel:
    file_size = config.fileSize  # in bits
    delay_budgets = config.delayBudgets
    min_data_rates = file_size / np.array(delay_budgets) / 1e6  # Mb/s
    num_apps = config.numApps
    app_names = config.appNames
    period = 60 * 60 * 24 * 7  # a week (in seconds)
    sample_rates = config.dpiSampleRates
    profiles_path = config.profilesPath
    _rng = np.random.default_rng()

    def __init__(self, area=None, period=period, scenario=None, sample_rate=None):
        self.area = 1 if area is None else area[0] * area[1] / 1e6  # km^2
        self.period = period
        self.scenario = scenario
        if sample_rate is None:
            if scenario is None:
                self.sample_rate = 1
            else:
                self.sample_rate = self.sample_rates[scenario.value - 1]
        else:
            self.sample_rate = sample_rate

    @classmethod
    def from_scenario(cls, scenario='RANDOM', **kwargs):
        if type(scenario) is str:
            s = scenario.upper()
            if s == 'RANDOM':
                s = cls._rng.choice(TrafficType._member_names_)
            scenario = TrafficType[s]
        else:
            scenario = TrafficType(scenario)
        profiles_df = pd.read_csv(cls.profiles_path, index_col=[0, 1, 2])
        profile = profiles_df.loc[int(scenario)]
        return cls(scenario=scenario, **kwargs).fit(profile)
    
    @classmethod
    def seed(cls, seed):
        cls._rng = np.random.default_rng(seed)
        
    def fit(self, data_rate_df, smoothen=True):
        """ Fit the traffic model to the given traffic trace dataset. """
        assert data_rate_df.shape[1] == self.num_apps
        df = data_rate_df[self.app_names] / self.sample_rate
        # smoothen and conv_smoothen(df)
        self.interval, rem = divmod(self.period, len(df))
        assert rem == 0, (self.interval, rem)
        
        self.rates = df * self.area / self.file_size  # files / s
        assert self.rates.values.max() * 1e-3 <= 1.

        self.densities = df / 1e6
        df = pd.concat([self.densities, self.rates], axis=1, keys=['Mb/s/km^2', 'files/s'])
        total_df = df.groupby(level=0, axis=1).sum()
        for k in df.columns.levels[0]:
            df[k, 'Total'] = total_df[k]
        info_df = df.describe().T[['mean', 'std', 'min', 'max']].sort_index()
        info_df['peak time'] = df.idxmax(axis=0).map(lambda x: f'{x[0]}, {x[1]}')
        info_df['vale time'] = df.idxmin(axis=0).map(lambda x: f'{x[0]}, {x[1]}')
        self.info_df = info_df.set_index(pd.MultiIndex.from_tuples(info_df.index))
        self.density_mean, self.density_std = self.info_df.loc[
            ('Mb/s/km^2', 'Total'), ['mean', 'std']]

        return self

    def print_info(self):
        notice('Traffic scenario: %s' % self.scenario.name)
        notice('%s\n' % self.info_df)
        
    @property
    def time_slots(self):
        return self.rates.index

    @timeit
    def emit_traffic(self, time, dt):
        """ Randomly generate traffic for the given time.
        
        Returns:
        A list of length n_apps, each element is either (traffic-demand, delay-budget) or False.
        """
        return [(self.file_size, delay) if np.random.rand() < p else (None, None)
                for p, delay in zip(self.get_arrival_rates(time, dt), self.delay_budgets)]

    def _get_time_loc(self, time):
        return int((time / self.period) % 1 * len(self.rates))
    
    def get_time_slot(self, time):
        i = self._get_time_loc(time)
        return self.time_slots[i]
    
    def get_start_time_of_slot(self, time_slot: str):
        m = re.match(r'(\w{3,}),?\s*(?:(\d+)(?::(\d+))?)?', time_slot)
        d, h, m = m[1][:3], int(m[2]) if m[2] else 0, int(m[3]) if m[3] else 0
        s = d, f"{h:02d}:{m:02d}"
        return self.interval * self.time_slots.get_loc(s)
    
    def get_arrival_rates(self, time, dt):
        i = self._get_time_loc(time)
        return self.rates.values[i] * dt  # (n_apps)


def conv_smoothen(df: pd.DataFrame, weights=(0.5, 1, 0.5), inplace=True):
    print(df.values.shape)
    pad = (len(weights) - 1) // 2
    a = np.pad(list(df.values), ((pad, pad), (0, 0)), mode='edge')
    a = np.apply_along_axis(np.convolve, 0, a, weights, mode='valid')
    if not inplace: df = df.copy()
    df[:] = a
    return df


if __name__ == '__main__':
    import plotly.express as px
    from plotly.subplots import make_subplots
    from collections import defaultdict

    figs = defaultdict(lambda: make_subplots(rows=len(TrafficType), cols=1, shared_xaxes=True))
    for i, traffic_type in enumerate(TrafficType):
        print(traffic_type)
        model = TrafficModel.from_scenario(traffic_type, sample_rate=1/300)
        model.print_info()
        for cat, rates in model.densities.items():
            days_idx = rates.index.get_level_values(0).unique()
            df = rates.unstack().reindex(days_idx)
            fig = px.imshow(df, title=traffic_type.name, labels=dict(x='time of day', y='day of week', color='Mb/s/km²'))
            print(cat)
            fig.show()
            figs[cat].add_trace(fig.data[0], row=i+1, col=1)
        print()
    for cat, fig in figs.items():
        fig.update_layout(height=600, width=600, title_text=cat)
        # fig.show()

TrafficType.A
Traffic scenario: A
                                 mean        std        min         max  \
Mb/s/km^2 Delay Sensitive   57.869375  19.304751  18.641390  102.714874   
          Delay Stringent    3.685933   2.490327   0.063344   12.350262   
          Delay Tolerant   113.430591  30.044876  48.960678  165.167608   
          Total            174.985900  49.644272  76.084361  262.538907   
files/s   Delay Sensitive   18.396179   6.136815   5.925938   32.652179   
          Delay Stringent    1.171727   0.791654   0.020136    3.926043   
          Delay Tolerant    36.058614   9.551009  15.564180   52.505369   
          Total             55.626519  15.781489  24.186567   83.458871   

                            peak time   vale time  
Mb/s/km^2 Delay Sensitive  Sun, 20:00  Fri, 02:00  
          Delay Stringent  Mon, 08:00  Sat, 03:20  
          Delay Tolerant   Sat, 14:40  Sat, 02:40  
          Total            Sun, 18:00  Mon, 03:00  
files/s   Delay Sensitive  Sun

Delay Sensitive


Delay Tolerant



TrafficType.B
Traffic scenario: B
                                 mean         std         min         max  \
Mb/s/km^2 Delay Sensitive   99.957603   36.368042   31.942635  169.305976   
          Delay Stringent    3.664691    2.509860    0.087902   10.294256   
          Delay Tolerant   210.821719   71.430786   72.273903  327.576509   
          Total            314.444013  106.796010  110.306918  469.969759   
files/s   Delay Sensitive   31.775666   11.561089   10.154290   53.820920   
          Delay Stringent    1.164974    0.797863    0.027943    3.272456   
          Delay Tolerant    67.018420   22.707235   22.975255  104.133768   
          Total             99.959060   33.949537   35.065625  149.399363   

                            peak time   vale time  
Mb/s/km^2 Delay Sensitive  Sun, 18:00  Tue, 02:20  
          Delay Stringent  Mon, 08:00  Sun, 04:40  
          Delay Tolerant   Mon, 15:00  Sat, 02:40  
          Total            Sun, 18:00  Sat, 02:40  
files/s   D

Delay Sensitive


Delay Tolerant



TrafficType.C
Traffic scenario: C
                                 mean         std        min         max  \
Mb/s/km^2 Delay Sensitive   89.594089   39.345249  19.980762  190.101096   
          Delay Stringent    3.018335    2.354277   0.012687   11.142265   
          Delay Tolerant   176.724041  101.730345  45.017479  409.206435   
          Total            269.336464  140.949110  69.851060  584.829009   
files/s   Delay Sensitive   28.481194   12.507518   6.351713   60.431511   
          Delay Stringent    0.959503    0.748405   0.004033    3.542031   
          Delay Tolerant    56.179060   32.339206  14.310671  130.083222   
          Total             85.619756   44.806515  22.205054  185.912135   

                            peak time   vale time  
Mb/s/km^2 Delay Sensitive  Fri, 14:00  Thu, 03:20  
          Delay Stringent  Tue, 08:00  Tue, 03:00  
          Delay Tolerant   Mon, 12:00  Mon, 03:00  
          Total            Mon, 11:00  Thu, 03:20  
files/s   Delay Sens

Delay Sensitive


Delay Tolerant
